In [6]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.autograd import Variable

import torchaudio
import sys
sys.path.append("..")
from dataset_generation.randomDataset import RandomDataset
from torch.utils.data import DataLoader
from utils.codec import CodecTransform
from utils.numpyDataset import NumpyDataset
from utils.cnn import CNNNetwork

In [ ]:
class DenoisingAutoencoder(nn.Module):
    def __init__(self):
        super(DenoisingAutoencoder, self).__init__()
        
        self.encoder = nn.Sequential(
            nn.Linear(8 * 300, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU()
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024, 8 * 300),
        )
        
    def forward(self, x):
        shape = x.shape
        
        x = x.view(x.size(0), -1)
        x = self.encoder(x)
        x = self.decoder(x)
        x = x.reshape(shape)
        return x

model = DenoisingAutoencoder()
model(torch.rand(64, 1, 8, 300)).shape

In [ ]:
dataset = NumpyDataset("./noisetrain40X", "./noisetrain40Y")
device = "cuda" if torch.cuda.is_available() else "cpu"

batch_size = 64
lr = 0.0001

v, t = torch.utils.data.random_split(dataset, [0.8, 0.2])
tdl = torch.utils.data.DataLoader(t, batch_size = batch_size)
vdl = torch.udtils.data.DataLoader(v, batch_size = batch_size)
model = DenoisingAutoencoder().to(device)

loss = torch.nn.CrossEntropyLoss()
opt = torch.optim.Adam(model.parameters(), lr=lr)

In [8]:
def train_epoch(model, train_data_loader, val_data_loader, loss_fn, opt, device, metrics):

    model.train()
    losses = []
    for input, target in train_data_loader:
        # disregard labels for autoencoder reconstruction loss

        input = input.unsqueeze(1)
        input = input.to(device)

        # calculate loss
        prediction = model(input)
        loss = loss_fn(prediction, target)
        losses.append(loss.item())

        # backpropagate error and update weights
        opt.zero_grad()
        loss.backward()
        # torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.1)
        opt.step()
    
    metrics['t_loss'].append((sum(losses)/len(losses)))
    
    model.eval()
    losses = []
    for input, target in val_data_loader:
        
        input = input.unsqueeze(1)
        input, target = input.to(device), target.to(device)
        prediction = model(input)

        loss = loss_fn(prediction, target)
        losses.append(loss.item())
    
    metrics['v_loss'].append((sum(losses)/len(losses)))

In [ ]:
metrics = {
    't_loss': [],
    'v_loss': [],
}
print('****** STARTING TRAINING ******')
for epoch in range(100):
    train_epoch(model, tdl, vdl, loss_fn=loss, opt=opt, device=device, metrics=metrics)
    print(f"Finished epoch: {epoch}; Training loss: {metrics['t_loss'][-1]}; Validation loss: {metrics['v_loss'][-1]}")

print("Fininshed training")    

plt.plot(metrics['t_loss'])
plt.plot(metrics['v_loss'])
plt.legend(['training', 'validation'])
plt.title('cross entropy loss')
plt.show()